In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col

# Create a SparkSession
spark = SparkSession.builder.appName("Jupyter").getOrCreate()

# Read the events CSV file and create a new column 'event_date'
events = spark.read.option("header", "true").csv("/home/iceberg/data/events.csv").withColumn("event_date", expr("DATE_TRUNC('day', event_time)"))

# Read the devices CSV file
devices = spark.read.option("header", "true").csv("/home/iceberg/data/devices.csv")

# Join the events and devices DataFrames
joined_df = events.join(devices, on="device_id", how="left")

# Rename columns using select and cast 'event_time' correctly
df = joined_df.select(
    "url",
    "referrer",
    col("browser_type").alias("browser_family"),
    col("os_type").alias("os_family"),
    col("device_type").alias("device_family"),
    "host",
    "event_time",
    "event_date"
)

# Show the result
df.show()

ConnectionRefusedError: [Errno 111] Connection refused

In [2]:
sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date"), col("host"), col("browser_family")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

sorted.show()

+--------------------+--------------------+--------------+---------+------------------+--------------------+--------------------+-------------------+
|                 url|            referrer|browser_family|os_family|     device_family|                host|          event_time|         event_date|
+--------------------+--------------------+--------------+---------+------------------+--------------------+--------------------+-------------------+
|                   /|                NULL|         Other|    Other|             Other|admin.zachwilson....|2021-01-07 09:21:...|2021-01-07 00:00:00|
|                   /|                NULL|         Other|    Other|             Other|    www.eczachly.com|2021-01-07 18:45:...|2021-01-07 00:00:00|
|                   /|                NULL|         Other|    Other|             Other|    www.eczachly.com|2021-01-07 21:57:...|2021-01-07 00:00:00|
|                   /|                NULL|      PetalBot|  Android|Generic Smartphone|    www.eczac

In [3]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [4]:
%%sql

DROP TABLE IF EXISTS bootcamp.eventsdevices

++
||
++
++

In [5]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.eventsdevices (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [6]:
# DataFrame with data to be inserted
sorted.write.format("iceberg").mode("append").saveAsTable("bootcamp.eventsdevices")

In [7]:
%%sql
select * from bootcamp.eventsdevices limit 5;

url,referrer,browser_family,os_family,device_family,host,event_time,event_date
/,None,Android,Android,Samsung SM-N900T,admin.zachwilson.tech,2023-01-01 17:14:00.071000,2023-01-01
/,None,Android,Android,ZTE BA520,admin.zachwilson.tech,2023-01-01 17:16:47.060000,2023-01-01
/,None,Android,Android,ZTE BA520,admin.zachwilson.tech,2023-01-01 17:16:52.225000,2023-01-01
/,None,Android,Android,ZTE BA520,admin.zachwilson.tech,2023-01-01 17:38:40.362000,2023-01-01
/,http://admin.zachwilson.tech,Chrome,Mac OS X,Other,admin.zachwilson.tech,2023-01-01 00:22:11.594000,2023-01-01


In [8]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [9]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg;

++
||
++
++

In [10]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    

first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_family"), col("host"))

sorted = df.repartition(10, col("event_date")) \
        .sortWithinPartitions(col("event_date")) \
        .withColumn("event_time", col("event_time").cast("timestamp")) \

start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [11]:
%%sql
SELECT *
FROM demo.bootcamp.events_sorted.files

content,file_path,file_format,spec_id,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/bootcamp/events_sorted/data/00000-59-f77f61e2-0404-4b1a-be66-9d9e6657e391-0-00001.parquet,PARQUET,0,89391,643178,"{1: 107448, 2: 61005, 3: 11437, 4: 12908, 5: 10692, 6: 7365, 7: 426434, 8: 2274}","{1: 89391, 2: 89391, 3: 89391, 4: 89391, 5: 89391, 6: 89391, 7: 89391, 8: 89391}","{1: 0, 2: 46359, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}",{},"{1: bytearray(b'/'), 2: bytearray(b'52.20.78.240'), 3: bytearray(b'%E3%82%A6%E3%82%'), 4: bytearray(b'Android'), 5: bytearray(b'17MB150WB'), 6: bytearray(b'aashish.techcrea'), 7: bytearray(b' \xba\xe7\xb8\xa8\xb8\x05\x00'), 8: bytearray(b'\x00\xa0&\xb4\xa8\xb8\x05\x00')}","{1: bytearray(b'/zzageqnf.php?Fp'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 5: bytearray(b'vivo $2'), 6: bytearray(b'zachwilson.techd'), 7: bytearray(b'\xe8\xb0\x1b\x8ec\x03\x06\x00'), 8: bytearray(b'\x00\xe0dqO\x03\x06\x00')}",None,[4],None,0,"Row(browser_family=Row(column_size=11437, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='%E3%82%A6%E3%82%', upper_bound='webprosbot'), device_family=Row(column_size=10692, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='17MB150WB', upper_bound='vivo $2'), event_date=Row(column_size=2274, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 0), upper_bound=datetime.datetime(2023, 8, 20, 0, 0)), event_time=Row(column_size=426434, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 1, 19, 764000), upper_bound=datetime.datetime(2023, 8, 20, 23, 59, 41, 89000)), host=Row(column_size=7365, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='aashish.techcrea', upper_bound='zachwilson.techd'), os_family=Row(column_size=12908, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='Android', upper_bound='iOS'), referrer=Row(column_size=61005, value_count=89391, null_value_count=46359, nan_value_count=None, lower_bound='52.20.78.240', upper_bound='zachwilson.tech'), url=Row(column_size=107448, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='/', upper_bound='/zzageqnf.php?Fp'))"
0,s3://warehouse/bootcamp/events_sorted/data/00001-60-f77f61e2-0404-4b1a-be66-9d9e6657e391-0-00001.parquet,PARQUET,0,99232,740373,"{1: 142161, 2: 67344, 3: 11896, 4: 16525, 5: 11505, 6: 9100, 7: 475847, 8: 2355}","{1: 99232, 2: 99232, 3: 99232, 4: 99232, 5: 99232, 6: 99232, 7: 99232, 8: 99232}","{1: 0, 2: 49299, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0}",{},"{1: bytearray(b'""/?""""<?=print(93'), 2: bytearray(b'""https://www.goo'), 3: bytearray(b') Bot'), 4: bytearray(b'Android'), 5: bytearray(b'13 Pro Max'), 6: bytearray(b'abhishekanand.te'), 7: bytearray(b'(\x83\xb2EX\xb8\x05\x00'), 8: bytearray(b'\x00 \xc9<X\xb8\x05\x00')}","{1: bytearray(b'/zz.php'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 5: bytearray(b'vivo $2'), 6: bytearray(b'zsavi524.techcrf'), 7: bytearray(b'\x88\xb8\x07P;\x03\x06\x00'), 8: bytearray(b""\x00 \xb65\'\x03\x06\x00"")}",None,[4],None,0,"Row(browser_family=Row(column_size=11896, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound=') Bot', upper_bound='webprosbot'), device_family=Row(column_size=11505, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound='13 Pro Max', upper_bound='vivo $2'), event_date=Row(column_size=2355, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 8, 0, 0), upper_bound=datetime.datetime(2023, 8, 18, 0, 0)), event_time=Row(column_size=475847, value_count=99232, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 8

In [12]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files


size,num_files,sorted
3301248,4,sorted
3589113,4,unsorted


In [13]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.eventsdevices.files;

size,num_files
3269744,3


In [14]:
%%sql 
SELECT COUNT(1) as num_files FROM bootcamp.matches_bucketed.files

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bootcamp`.`matches_bucketed`.`files` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 34;
'Aggregate [count(1) AS num_files#480L]
+- 'UnresolvedRelation [bootcamp, matches_bucketed, files], [], false
